# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [6]:
data.tail()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
4865,b,99,3,2,1,0,0,0,1,313,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,Private
4866,a,99b,4,4,6,0,0,0,0,285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
4867,a,99b,4,6,8,0,1,0,0,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
4868,a,99b,4,4,2,0,1,1,0,267,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,
4869,a,99b,4,3,7,0,0,0,1,274,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,


Question 1: What test is appropriate for this problem? Does CLT apply?

The question here is find out if there is a difference in the average call rate between White race applicants 
and other race applicants. So, we are comparing the mean of two sample, where the population standard deviation σ is not known. The sample size, n=4869 is > 30. We will use a two sample t test to solve this. Yes, Central Limit Theorem does apply.

Question 2:What are the null and alternate hypotheses?
Ho: That there is no difference is call back staus of black and white label resumes based on names.
H1: That there is a statical difference in those who get called back for additional interviews simply based on name

In [7]:
w = data[data.race=='w']
b = data[data.race=='b']

In [14]:
len(w)

2435

In [15]:
len(b)

2435

In [38]:
# Your solution to Q3 here
#Compute margin of error, confidence interval, and p-value. 
#Try using both the bootstrapping and the frequentist statistical approaches.
w_cb = w.call
b_cb = b.call

# Calculating margin of error, confidence interval and p-value using bootstrapping approach
def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data1, data2, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(shape=size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data1, func) - bootstrap_replicate_1d(data2, func)

    return bs_replicates

# Draw bootstrap replicates of the mean difference between the frequencies of calls for resumes for black-sounding and white-sounding names: bs_replicates
print("Statistical inference using the bootstrap approach:")
bs_replicates = draw_bs_reps(w.call, b.call, np.mean, 10000)

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Print the confidence interval
print("The 95% confidence interval:" + str(conf_int))


def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(shape=size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1, data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1, data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# Compute difference of mean impact force from experiment: empirical_diff_means
empirical_diff_means = diff_of_means(w.call, b.call)

# Under the null hypothesis there is no difference between the frequency of the calls between white-sounding and black-sounding names
# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call,
                                 diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

# Print the result
print("The p-value:"+ str(p))

w_mean= np.mean(w_cb)
b_mean= np.mean(b_cb)
print("mean for white-sounding names call-backs are " + str(w_mean)) 
print("mean for black- sounding names call-backs are " + str(b_mean))

diff_mean = w_mean - b_mean
diff_sd = np.sqrt(w_mean/w.shape[0] + b_mean/b.shape[0])
z_critical = stats.norm.ppf(.975)

# Using 95% confidence for the margin of error and the confidence interval
margin_of_error_CLT = z_critical * diff_sd
confidence_interval_CLT = diff_mean - margin_of_error_CLT, diff_mean + margin_of_error_CLT

# Using 0.05 confidence level for the p-value
p_value_CLT = stats.norm.cdf(-diff_mean/(diff_sd)) * 2

print("Statistical inference using frequentist statistical approaches:")
print("The margin of error for the 95% confidence interval:" + str(margin_of_error_CLT))
print("The 95% confidence interval: " + str(confidence_interval_CLT))
print("The p-value: " + str(p_value_CLT))



Statistical inference using the bootstrap approach:
The 95% confidence interval:[0.01683778 0.04722793]
The p-value:0.0
mean for white-sounding names call-backs are 0.09650924056768417
mean for black- sounding names call-backs are 0.0644763857126236
Statistical inference using frequentist statistical approaches:
The margin of error for the 95% confidence interval:0.0159364710806662
The 95% confidence interval: (0.016096383774394377, 0.04796932593572678)
The p-value: 8.161927657237071e-05


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>
4) Write a story describing the statistical significance in the context or the original problem.
Looking at the stats now properly calculated it is evidant to see that there is indeed a bias for hiring white sounding names over black sounding names. The other bit of evidance that would also be significant but is missing is if the employer that was looking at this application was either white or POC. This could also create a bias that would be notable to look at. 

5)Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
No, this is strictly looking at the bias of names, the other factors that we'd need to consider before entering a report are years of education and years of experience, which would vastly weed out or promote peoples to the next round. This is simply one test of this data. 